# Introduction
[Chainer](http://chainer.org/) とはニューラルネットの実装を簡単にしたフレームワークです。

* 今回は対話にニューラルネットを適用してみました。

![](./pictures/Chainer.jpg)

* 今回は対話モデルの構築を行っていただきます。


対話モデルは人間の発言に対して応答して返すモデルです。

対話モデルにはいくつか種類があるのでここでも紹介しておきます。

* POMDP
 * [POMDP](https://github.com/pybrain/pybrain/blob/master/pybrain/rl/environments/mazes/tasks/pomdp.py)対話モデルの一般的な機械学習のモデルです。
* End TO End ニューラル対話モデル
 * 入力をユーザーの入力、出力をシステム側の出力として機械翻訳と同じ枠組みで対応しているモデル

以下では、このChainerを利用しデータを準備するところから実際にNN対話モデルを構築し学習・評価を行うまでの手順を解説します。

<A HREF=#1.各種ライブラリ導入 >1.各種ライブラリ導入</A><br>
<A HREF=#2.対話のクラス >2.対話のクラス</A><br>
<A HREF=#3.各値を設定 >3.各値を設定</A><br>
<A HREF=#4.実行 >4.実行</A><br>
<A HREF=#5.学習したモデルの動作テスト >5.学習したモデルの動作テスト</A><br>

##  <A NAME=1.各種ライブラリ導入 /> 1.各種ライブラリ導入

Chainerの言語処理では多数のライブラリを導入します。
Ctrl → m → lをコードの部分で入力すると行番号が出ます。ハンズオンの都合上、行番号があった方が良いので対応よろしくお願いします。

In [ ]:
#表示用に使用しています。
from util.functions import trace
from util.functions import read_data
from util.functions import read_image

import numpy as np

from chainer import Chain, Variable, cuda, functions, links, optimizer, optimizers, serializers
import chainer.links as L

from EncoderDecoderModel import EncoderDecoderModel
import subprocess

`導入するライブラリの代表例は下記です。

* `numpy`: 行列計算などの複雑な計算を行なうライブラリ
* `chainer`: Chainerの導入
* `util`:今回の処理で必要なライブラリが入っています。
* `w2v_model`:初期値の設定にWord2vecを使用して初期値の最適化を行なっています。


##  <A NAME=2.対話のクラス /> 2.対話のクラス

下記の論文を参考にしてforward処理を記述しています。

http://arxiv.org/pdf/1507.04808.pdf


下記を設定しています。
* ニューラルネットを用いて対話用のモデルを構成しています。

全体構成

![](./pictures/NN_machine_translation.png)




##  <A NAME=3.各値を設定 /> 3.各値を設定

各値を設定

* ユーザーの発言の設定(学習データ)
* システム応答結果の設定（学習データ）
* ユーザーの発言の設定(テストデータ)
* システム応答結果の設定（テストデータ）
* 語彙の設定
* 潜在空間の設定
* 隠れ層の設定
* 学習回数の設定
* ミニバッチサイズの設定
* 最大予測言語数の設定
ベストな調整方法は経験則か力技です。グリッドサーチ、ランダムサーチ、データから推定など。

In [ ]:
parameter_dict = {}
train_path = "data/"
parameter_dict["source"] = train_path + "player_1_wakati"
parameter_dict["target"] = train_path + "player_2_wakati"
parameter_dict["test_source"] = train_path + "player_1_wakati"
parameter_dict["test_target"] = train_path + "player_2_test"
#--------Hands on  2----------------------------------------------------------------

"""
下記の値が大きいほど扱える語彙の数が増えて表現力が上がるが計算量が爆発的に増えるので大きくしない方が良いです。
"""
parameter_dict["vocab"] = 5000

"""
この数が多くなればなるほどモデルが複雑になります。この数を多くすると必然的に学習回数を多くしないと学習は
収束しません。
語彙数よりユニット数の数が多いと潜在空間への写像が出来ていないことになり結果的に意味がない処理になります。
"""
parameter_dict["embed"] = 300

"""
この数も多くなればなるほどモデルが複雑になります。この数を多くすると必然的に学習回数を多くしないと学習は
収束しません。
"""
parameter_dict["hidden"] = 500

"""
学習回数。基本的に大きい方が良いが大きすぎると収束しないです。
"""
parameter_dict["epoch"] = 20

"""
ミニバッチ学習で扱うサイズです。この点は経験的に調整する場合が多いが、基本的に大きくすると学習精度が向上する
代わりに学習スピードが落ち、小さくすると学習精度が低下する代わりに学習スピードが早くなります。
"""
parameter_dict["minibatch"] = 64

"""
予測の際に必要な単語数の設定。長いほど多くの単語の翻訳が確認できるが、一般的にニューラル翻訳は長い翻訳には
向いていないので小さい数値がオススメです。
"""
parameter_dict["generation_limit"] = 256

#--------Hands on  2----------------------------------------------------------------#

##  <A NAME=4.実行 /> 4.実行

In [ ]:
trace('initializing ...')

encoderDecoderModel = EncoderDecoderModelForward(parameter_dict)
encoderDecoderModel.train()